# Goal:
Create a web app that analyzes and synthesizes 2 users' spotify recent taste profiles so that we can create a playlist filled with brand new song recommendations for both of them to discover together. The playlist we curate will automatically end up in both of their spotify libraries. Would be a cool new way for people to socialize because the playlist is unique to them and can create a special experience for both to find new music together that matches their music tastes.

### Some useful links to get started: 

spotipy documentation: https://spotipy.readthedocs.io/en/2.13.0/ 

github of guy who made this code (give proper credit later): https://github.com/makispl/Spotify-Data-Analysis

he also has a useful medium article that gives steps on how to use spotify api and this code: https://towardsdatascience.com/can-a-data-scientist-replace-a-dj-spotify-manipulation-with-python-fbbd4a45ffd5

I've also created an app on spotify developers already and this is the client id: 0b2103231ba64a70885c27fbb38cfa97
and this is the client secret: 45d7d10e11474298abcb52df745e2b25

### plan
get top 50 from each user -> k=5 cluster 100 songs -> put 5 seed_songs/artists/genres into recommendations() function parameters -> keep calling function until 30 new songs that are not in either user's listen history

In [1]:
# Import the libraries
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

## Authorization Flow

In [2]:
# Declare the credentials
cid = '0b2103231ba64a70885c27fbb38cfa97'
secret = '45d7d10e11474298abcb52df745e2b25'
redirect_uri='http://localhost:7777/callback'
username = 'y'

In [3]:
# Authorization flow
scope = 'user-top-read'
token = util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

## Extract User's Top Medium Term Songs

In [4]:
# Fetch the top-50 songs of the user (medium term)
if token:
    sp = spotipy.Spotify(auth=token)
    results = sp.current_user_top_tracks(limit=50,offset=0,time_range='medium_term')
    for song in range(50):
        list1 = []
        list1.append(results)
        with open('top50_data1.json', 'w', encoding='utf-8') as f:
            json.dump(list1, f, ensure_ascii=False, indent=4)
else:
    print("Can't get token for", username1)
    
# Open the JSON file to Python objects
with open('top50_data.json') as f:
  data = json.load(f)

print(len(data[0]['items']))
    
# Open the JSON file to Python objects
with open('top50_data2.json') as g:
  data2 = json.load(g)

print(len(data2[0]['items']))

50
50


In [5]:
list_of_results = data[0]["items"]
list_of_artist_names = []
list_of_artist_uri = []
list_of_song_names = []
list_of_song_uri = []
list_of_durations_ms = []
list_of_explicit = []
list_of_albums = []
list_of_popularity = []

for result in list_of_results:
    result["album"]
    this_artists_name = result["artists"][0]["name"]
    list_of_artist_names.append(this_artists_name)
    this_artists_uri = result["artists"][0]["uri"]
    list_of_artist_uri.append(this_artists_uri)
    list_of_songs = result["name"]
    list_of_song_names.append(list_of_songs)
    song_uri = result["uri"]
    list_of_song_uri.append(song_uri)
    list_of_duration = result["duration_ms"]
    list_of_durations_ms.append(list_of_duration)
    song_explicit = result["explicit"]
    list_of_explicit.append(song_explicit)
    this_album = result["album"]["name"]
    list_of_albums.append(this_album)
    song_popularity = result["popularity"]
    list_of_popularity.append(song_popularity)

In [6]:
# Convert the pulled content to a pandas df
all_songs = pd.DataFrame(
    {'artist': list_of_artist_names,
     'artist_uri': list_of_artist_uri,
     'song': list_of_song_names,
     'song_uri': list_of_song_uri,
     'duration_ms': list_of_durations_ms,
     'explicit': list_of_explicit,
     'album': list_of_albums,
     'popularity': list_of_popularity
     
    })

In [7]:
all_songs.sort_values('popularity', ascending=False).head(20)

,artist,artist_uri,song,song_uri,duration_ms,explicit,album,popularity
25,StaySolidRocky,spotify:artist:1XLWox9w1Yvbodui0SRhUQ,Party Girl,spotify:track:5RqR4ZCCKJDcBLIn4sih9l,147800,False,Party Girl,94
43,Drake,spotify:artist:3TVXtAsR1Inumwj472S9r4,Toosie Slide,spotify:track:127QTOFJsJQp5LbJbu3A1y,247058,True,Toosie Slide,93
8,BENEE,spotify:artist:0Cp8WN4V8Tu4QJQwCN5Md4,Supalonely,spotify:track:4nK5YrxbMGZstTLbvj6Gxw,223480,True,STELLA & STEVE,93
22,Roddy Ricch,spotify:artist:757aE44tKEUQEqRuT6GnEB,The Box,spotify:track:0nbXyq5TXYPCO7pr3N8S4I,196652,True,Please Excuse Me For Being Antisocial,93
15,Dua Lipa,spotify:artist:6M2wZ9GZgrQXHCFfjv46we,Break My Heart,spotify:track:1raaNykBg1bDnWENUiglUA,221820,False,Break My Heart,88
1,SAINt JHN,spotify:artist:0H39MdGGX6dbnnQPt6NQkZ,Roses - Imanbek Remix,spotify:track:7fPuWrlpwDcHm5aHCH5D9t,176218,True,Roses (Imanbek Remix),85
23,Tame Impala,spotify:artist:5INjqkS1o8h1imAzPqGZBb,The Less I Know The Better,spotify:track:6K4t31amVTZDgR3sKmwUJJ,216320,True,Currents,85
27,Gunna,spotify:artist:2hlmm7s2ICUX0LVIhVFlZQ,DOLLAZ ON MY HEAD (feat. Young Thug),spotify:track:3nS9a01VvXHQriLqJYwRqG,197760,True,WUNNA,85
3,BROCKHAMPTON,spotify:artist:1Bl6wpkWCQ4KVgnASpvzzA,SUGAR,spotify:track:6U0FIYXCQ3TGrk4tFpLrEA,204533,True,GINGER,83
2,WILLOW,spotify:artist:3rWZHrfrsPBxVy692yAIxF,Wait a Minute!,spotify:track:0y60itmpH0aPKsFiGxmtnh,196520,False,ARDIPITHECUS,83


In [8]:
#Making into A CSV

all_songs.to_csv('User2_top_50_songs.csv') 